<a href="https://colab.research.google.com/github/Evans-tats/machine-learning/blob/main/fine_tuning_a_model_with_trainer_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [30]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [31]:
from transformers import TrainingArguments
train_args = TrainingArguments("test-trainer")

In [7]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from transformers import Trainer
trainer = Trainer(
    model,
    train_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [9]:
trainer.train()

Step,Training Loss
500,0.539200
1000,0.304300


TrainOutput(global_step=1377, training_loss=0.3535851483424678, metrics={'train_runtime': 226.6814, 'train_samples_per_second': 48.544, 'train_steps_per_second': 6.075, 'total_flos': 405114969714960.0, 'train_loss': 0.3535851483424678, 'epoch': 3.0})

In [37]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)
predictions.predictions[2]

In [22]:
import numpy as np
pred = np.argmax(predictions.predictions, axis=-1)
pred[2]

0

In [23]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=pred, references=predictions.label_ids)

{'accuracy': 0.8602941176470589, 'f1': 0.900523560209424}

In [33]:
def compute_metrics(eval_preds):
  metric = evaluate.load("glue","mrpc")
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)


In [35]:
training_args = TrainingArguments("test_trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.359554,0.852941,0.895833
2,0.506300,0.498617,0.865196,0.905983
3,0.269400,0.606891,0.879902,0.915663


TrainOutput(global_step=1377, training_loss=0.3144838448097157, metrics={'train_runtime': 221.7733, 'train_samples_per_second': 49.618, 'train_steps_per_second': 6.209, 'total_flos': 405114969714960.0, 'train_loss': 0.3144838448097157, 'epoch': 3.0})